In [1]:
#import libraries
import db_ops
import pandas as pd

Connecting to the PostgreSQL database...
Connection successful


## Load file, parse csv and fix some of the problems

In [2]:
'''
# Read csv into panda, do not infer column names
invoice_lines = pd.read_csv("2023_08 invoices.csv", encoding = "ISO-8859-1", header = None, index_col = 0, sep="\t")

# Read csv into panda, just to get the column names
df_tmp = pd.read_csv("2023_08 invoices.csv", encoding = "ISO-8859-1", index_col = 0, sep="\t")
invoice_lines.columns = df_tmp.columns
invoice_lines.columns
'''

'\n# Read csv into panda, do not infer column names\ninvoice_lines = pd.read_csv("2023_08 invoices.csv", encoding = "ISO-8859-1", header = None, index_col = 0, sep="\t")\n\n# Read csv into panda, just to get the column names\ndf_tmp = pd.read_csv("2023_08 invoices.csv", encoding = "ISO-8859-1", index_col = 0, sep="\t")\ninvoice_lines.columns = df_tmp.columns\ninvoice_lines.columns\n'

## Brands table

In [9]:
df_brands = pd.DataFrame({'brand_id':[1,2,3,4,5],
                         'name':['Coca Cola', 'Corona', 'Miller', 'Budweiser', 'LoveIs'],
                         'score':[10,20,30,40,50]})
df_brands

,brand_id,name,score
0,1,Coca Cola,10
1,2,Corona,20
2,3,Miller,30
3,4,Budweiser,40
4,5,LoveIs,50


In [10]:
# Create stage table for new products 
db_ops.connection.rollback()
query = """

drop table if exists brand;

CREATE TABLE "brand"(brand_id SERIAL PRIMARY KEY
    , name VARCHAR(255)
    , score VARCHAR(255)
);

"""
db_ops.cursor.execute(query)
db_ops.connection.commit()

# Insert dataframe into table
db_ops.add_df(df_brands, "brand")

INSERT INTO brand (brand_id,name,score) VALUES ('1','Coca Cola','10');
INSERT INTO brand (brand_id,name,score) VALUES ('2','Corona','20');
INSERT INTO brand (brand_id,name,score) VALUES ('3','Miller','30');
INSERT INTO brand (brand_id,name,score) VALUES ('4','Budweiser','40');
INSERT INTO brand (brand_id,name,score) VALUES ('5','LoveIs','50');
